This notebook was made to collect the data in the UGC Fridge Portal through the STAC API.

In [18]:
import pystac_client as pc
import ipywidgets
import folium
from pystac import Item
import shapely
from shapely.geometry import Point, mapping
from shapely.geometry.polygon import Polygon
from pyproj import Transformer
from shapely.ops import transform
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import date
pd.set_option('display.max_columns', None)




%matplotlib inline

In [19]:
cat = pc.Client.open("https://stac.pgc.umn.edu/api/v1/")
collection = cat.get_collection("arcticdem-strips-s2s041-2m")
collection
collection = "arcticdem-strips-s2s041-2m"

In [36]:
# takes in coordinates, returns the max and min latitudes, and max and min longitudes based on radius.
# expects radius in meters, latitude and longitude in degrees
def bound_box(latitude, longitude, radius):
    km2deg = 1.0/111 # trick seen from https://stacspec.org/en/tutorials/access-sentinel-2-data-aws/
    r = radius * km2deg
    bbox = (latitude - r, longitude - r, longitude + r, latitude + r)
    return bbox

def init_catalog(url):
    catalog = pc.Client.open(url)
    return catalog

def init_collection(catalog):
    arctic_collection_name = "arcticdem-strips-s2s041-2m"
    collection = catalog.get_collection(arctic_collection_name)
    return collection

def pull_data(latitude, longitude):
    radius = 10000
    today = date.today()
    curr_year = today.year    

    catalog = init_catalog("https://stac.pgc.umn.edu/api/v1/")
    collection = init_collection(catalog)

    bbox = bound_box(latitude, longitude, radius)
    printed = False
    for year in range(2000, curr_year):
        search = catalog.search(
            collections = collection,
            bbox = bbox,
            datetime=[f'{year}-07-01T00:00:00Z', f'{year}-07-31T00:00:00Z']
            )
        if printed == False:
            printed = True
            items = search.items()
            print((items))



# longitude is x, latitude is y
def getData(lat, lon):
    
    df = None
    
    # this code generates a circle to query the data in
    # radius of circle in terms of meters
    radius = 10000

    # defines center of circle
    center = Point(lat, lon)
    
    # transforms from lat-lon to meters
    transform_to = Transformer.from_crs(4326, 3413, always_xy=True)
    projected = transform(transform_to.transform, center)
    
    # creates the circle with specified radius, higher resolution = smoother circle
    circle = projected.buffer(radius, resolution=500)
    
    # transforms from meters to lat-lon
    transform_back = Transformer.from_crs(3413, 4326, always_xy=True)
    projected = transform(transform_back.transform, circle)
    
    # saves the shape as a geojson
    geojson = shapely.to_geojson(projected)
    gdf = gpd.read_file(geojson)
    
    # filter out data
    for year in range(2008, 2025):

        search = cat.search(
        collections = collection,
        intersects = geojson,
        datetime=f"{year}-07-01/{year}-07-31")
        
        try:
            if df is None:
                df = gpd.GeoDataFrame.from_features(search.item_collection().to_dict(), crs="epsg:4326").to_crs(3413)
                items = list(search.items())
                df["item"] = items
                # print(year)
            else: 
                # print(year)
                new_df = gpd.GeoDataFrame.from_features(search.item_collection().to_dict(), crs="epsg:4326").to_crs(3413)
                items = list(search.items())
                new_df["item"] = items
                df = pd.concat([df, new_df], ignore_index=True)

        except ValueError:
            # print(f"None: {year}")
            pass
        
        try:
            validity_mask = df["pgc:valid_area_percent"] > 0.9
            df = df[validity_mask]
        except TypeError: 
            pass
    
    # visualizes queried data/circle AOI
    m = gdf.explore(name="AOI")
    m = df.explore(m=m, color='blue', name='Strips')
    folium.LayerControl().add_to(m)
    display(m)
    
    data = []
    try:
        for item in df["item"]:
            data.append(item.assets["dem"].href)
    except TypeError: 
            pass

    return data

    




In [37]:
coordinates = [[-42, 80], 
               [-60, 79], 
               [-35, 77], 
               [-55, 76], 
               [-42, 75], 
               [-25, 80], 
               [-36, 72.5], 
               [-32, 70], 
               [-39, 68], 
               [-45, 64], 
               [-48, 68]]
pull_data(coordinates[0][0], coordinates[0][1])
# for coordinate in other_coordinates:
#     getData(coordinate[0], coordinate[1])

<generator object ItemSearch.items at 0x0000021BFD4DC040>
